# Valheim Food EDA

In this notebook, I will analyze the food choices in the game, Valheim. From this analysis, I'd like to answer the following questions:
- Which food trio will give you the most HP / Stamina?
- Which food trio is best if you want a well-balanced stat?
- Which food trio is best for combat? For exploration?

## Import libraries

We will be using BeautifulSoup, Pandas, and Altair to scrape, clean, and visualize the data respectively.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import fandom
import re
import requests

In [2]:
# Set the wiki root to search for
fandom.set_wiki("Valheim")
# Set the URL
url = "https://valheim.fandom.com/wiki/Food"

## Data cleaning and preparation

It seems that the Valheim Wiki has compiled a list of food choices in the game. Good thing, pandas has a `read_html` function that allows me to get the table directly without having to write my own scraper. This function gives a list of DataFrame tables found in the given webpage.

In [3]:
df_list = pd.read_html(url, flavor="bs4")
df = df_list[0]  # We only have 1 table in that page, so let's just get that one 

In [4]:
df.head(5)

,Name,Icon,Health,Stamina,Total,Healing (hp/tick),Duration (sec),Stack Size
0,Raspberries,NaN,7,20,27.0,1,600,50
1,Mushroom,NaN,15,15,30.0,1,900,50
2,Blueberries,NaN,8,25,33.0,1,600,50
3,Carrot,NaN,10,32,42.0,1,900,50
4,Cloudberries,NaN,13,40,53.0,1,900,50


It looks good, it seems that I have most of the information I need (Health, Stamina, Duration). The remaining column that I need is the "Stage" of the game. I guess I'll need to find it by other means. In addition, the `Icon` column are all NaNs. I can get them by scraping it manually:

In [5]:
content = requests.get(url)
soup = BeautifulSoup(content.text)
image_links = [img["href"] for img in soup.find("table").find_all("a", class_="image", href=True)]

In [6]:
# Replace Icon with the links
df["Icon"] = image_links

In [7]:
df.head()

,Name,Icon,Health,Stamina,Total,Healing (hp/tick),Duration (sec),Stack Size
0,Raspberries,https://static.wikia.nocookie.net/valheim/imag...,7,20,27.0,1,600,50
1,Mushroom,https://static.wikia.nocookie.net/valheim/imag...,15,15,30.0,1,900,50
2,Blueberries,https://static.wikia.nocookie.net/valheim/imag...,8,25,33.0,1,600,50
3,Carrot,https://static.wikia.nocookie.net/valheim/imag...,10,32,42.0,1,900,50
4,Cloudberries,https://static.wikia.nocookie.net/valheim/imag...,13,40,53.0,1,900,50


I might also need to get some other information from the infobox of individual pages. If that's the case then I can use this neat library called [fandom-py](https://github.com/NikolajDanger/fandom-py) that easily allows me to obtain the information that I need. There's some bit of data wrangling involved, so let's start rolling up our sleeves!

In [13]:
from typing import Dict

def get_infobox(row) -> Dict:
    """Get relevant information from the infobox"""
    name = row["Name"]
    url = ""
    drop_by_data = ""
    craft_data = ""

    try:
        # Get page and its URL
        page = fandom.page(name)
        url = page.url
        
        # Get infobox since we need a lot of info there
        infobox = page.content["infobox"]
        
        # Split the infobox for easier parsing
        # We just need to get 'Dropped by' and 'Crafting Materials'
        l = infobox.split("\n")
        if "Dropped by" in l:
            drop_idx = l.index("Dropped by")
            drop_by_data = l[drop_idx + 1]
        if "Crafting Materials" in l:
            craft_idx = l.index("Crafting Materials")
            craft_data_ = l[craft_idx + 1]
            craft_data = re.sub(r'(?<=\d)(?=[^\d\s])|(?<=[^\d\s])(?=\d)', ' ', craft_data_)         
    except fandom.error.PageError:
        print(f"Cannot find page for {name}, will return empty strings")
    

    return {"URL": url, "DroppedBy": drop_by_data, "Crafting": craft_data}

What I did is that I obtained the Fandom page for each food item, then filtered it with just the infobox. However, the return value is just one long string delimited by `\n`, so my workaround is to split it with that value. To get the key value pairs, I just took the *next* item of a specific element, assuming that's how it works. For example, to get the value for the key "Dropped by," I looked at the element after it. It seems to work!

Let's now put the URL, Drop information, and Craft information in one Dataframe and concatenate it with the rest of the crew!

In [14]:
infobox = pd.DataFrame([get_infobox(row) for _, row in df.iterrows()])

Cannot find page for Bukeperries, will return empty strings


In [15]:
# Some items aren't craftable as you can just pick them up from the environment
infobox.head(10)

,URL,DroppedBy,Crafting
0,https://valheim.fandom.com/en/wiki/Raspberries,Raspberry bushes in Meadows,
1,https://valheim.fandom.com/en/wiki/Mushroom,"Meadows, Black Forest, Swamp",
2,https://valheim.fandom.com/en/wiki/Blueberries,Blueberry bushes in the Black Forest biome,
3,https://valheim.fandom.com/en/wiki/Carrot,Farming Carrot seeds,
4,https://valheim.fandom.com/en/wiki/Cloudberries,Plains biome,
5,https://valheim.fandom.com/en/wiki/Honey,Beehive,
6,https://valheim.fandom.com/en/wiki/Yellow_mush...,"Burial Chambers, Troll Cave, Sunken Crypts",
7,https://valheim.fandom.com/en/wiki/Carrot_soup,Cauldron,Mushroom x 1 Carrot x 3
8,https://valheim.fandom.com/en/wiki/Queens_jam,Cauldron,Crafts 4 Raspberries x 8 Blueberries x 6
9,https://valheim.fandom.com/en/wiki/Grilled_nec...,Cooking station,Neck tail x 1


In [19]:
master_df = pd.concat([df, infobox], axis=1)
master_df.head()

,Name,Icon,Health,Stamina,Total,Healing (hp/tick),Duration (sec),Stack Size,URL,DroppedBy,Crafting
0,Raspberries,https://static.wikia.nocookie.net/valheim/imag...,7,20,27.0,1,600,50,https://valheim.fandom.com/en/wiki/Raspberries,Raspberry bushes in Meadows,
1,Mushroom,https://static.wikia.nocookie.net/valheim/imag...,15,15,30.0,1,900,50,https://valheim.fandom.com/en/wiki/Mushroom,"Meadows, Black Forest, Swamp",
2,Blueberries,https://static.wikia.nocookie.net/valheim/imag...,8,25,33.0,1,600,50,https://valheim.fandom.com/en/wiki/Blueberries,Blueberry bushes in the Black Forest biome,
3,Carrot,https://static.wikia.nocookie.net/valheim/imag...,10,32,42.0,1,900,50,https://valheim.fandom.com/en/wiki/Carrot,Farming Carrot seeds,
4,Cloudberries,https://static.wikia.nocookie.net/valheim/imag...,13,40,53.0,1,900,50,https://valheim.fandom.com/en/wiki/Cloudberries,Plains biome,


## Quick visualization